# Vowpal Wabbit

Поработаем с той же задачей и с теми же данными, на которых мы настраивали параметры xgboost.

Посмотрим, справятся ли линейные алгоритмы, реализованные в vowpal wabbit, с этой задачей.

In [1]:
!wget https://www.dropbox.com/s/eg9wt98ak9n91w2/train_medium.csv?dl=0
!wget https://www.dropbox.com/s/mji0jvu5wo34p2w/test_medium.csv?dl=0


--2019-01-24 16:44:39--  https://www.dropbox.com/s/eg9wt98ak9n91w2/train_medium.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/eg9wt98ak9n91w2/train_medium.csv [переход]
--2019-01-24 16:44:40--  https://www.dropbox.com/s/raw/eg9wt98ak9n91w2/train_medium.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc2699d77833755c3f649584f97a.dl.dropboxusercontent.com/cd/0/inline/AaDwUUx3M9zUjTpudD1WN4nMU_NtMBz9A5czhDQIg0StTLfJ57f1AL51aMNDgQjI0VHX2ts-hJY1GSdxWbqK92ZU41UnfnhvVWf0t11J1oyfVg/file# [переход]
--2019-01-24 16:44:41--  https://uc2699d77833755c3f649584f97a.dl.dropboxusercontent.com/cd/0/inline/AaDwUUx3M9zUjTpudD1WN4nMU_NtMBz9A5czhDQIg0StTLfJ57f1AL51aMNDgQjI0VHX2ts-hJY1GSdxWbqK92ZU41UnfnhvVWf0t11J1oyfVg/file
Распознаётся uc26

In [3]:
!ls

Blending.pdf           test.vw                train_medium.csv
BlendingStacking.ipynb test_medium.csv        train_medium.csv?dl=0
GD.pdf                 test_medium.csv?dl=0   vw_tutorial.ipynb
README.md              train-sample.csv?dl=0  vw_tutorial.pdf
model.vw               train.vw               vw_vs_xgb.ipynb
pred.txt               train.vw.cache


In [4]:
import pandas as pd

X_train = pd.read_csv('train_medium.csv?dl=0')
X_test = pd.read_csv('test_medium.csv?dl=0')

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Измените функцию save_to_vw из предыдущего ноутбука (см.ниже) так, чтобы она записывала файлы train_medium и test_medium в нужном формате. Не забудьте, что столбец 'Disbursed' является целевым, поэтому а) он определяет значение переменной target, b) его не нужно записывать в файл-матрицу с признаками.

In [7]:
X_test.head()

,Unnamed: 0,Existing_EMI,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,Interest_Rate_Missing,...,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2,Disbursed
0,27487,0.0,300000.0,5.0,22000,2,3,28,1,1,...,0,0,0,0,0,1,0,0,1,0.0
1,56689,0.0,50000.0,1.0,20000,3,1,34,1,1,...,0,0,0,1,0,1,1,0,0,0.0
2,47624,0.0,0.0,0.0,5000,7,0,22,1,1,...,1,0,0,0,1,0,1,0,0,0.0
3,38700,11500.0,500000.0,4.0,200000,5,10,41,0,0,...,0,0,0,0,0,1,1,0,0,0.0
4,12905,8000.0,200000.0,4.0,20500,3,1,26,1,1,...,0,0,0,0,1,0,0,1,0,0.0


In [185]:
columns = X_train.columns
columns = list(columns)
columns.remove('Unnamed: 0')
columns.insert(0, 'Unnamed')

In [186]:
def trasform_to_string(tpl):
    result = str(tpl)[1:-1]
    result = result.replace("'", "")
    result = result.replace(", ", ":")
    return result

In [203]:
def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for row in data.iterrows():
            values = list(row[1])
            r = list(zip(columns, values))
            target = r.pop()[1]
            r = [trasform_to_string(i) for i in r]
            r = " ".join(r)
            fout.write('{0} | {1}| \n'.format(target, r))

In [204]:
save_to_vw(X_train, 'train_medium_vw.vw')
save_to_vw(X_test, 'test_medium_vw.vw')

Скопируйте полученные файлы в папку share и обучите vowpal wabbit. Например, предсказания классов можно сохранять в файл с именем preds.txt, а предсказания вероятностей - в файл probs.txt.

Поэкспериментируйте с параметрами vowpal wabbit и постарайтесь добиться наилучшего качества на тесте.

Постарайтесь, насколько это возможно, приблизить ROC-AUC к значению, полученному после оптимизации xgboost.

In [205]:
!ls

Blending.pdf           test_medium.csv        train_medium.csv?dl=0
BlendingStacking.ipynb test_medium.csv?dl=0   train_medium_vw.vw
GD.pdf                 test_medium_vw.vw      vw_tutorial.ipynb
README.md              train-sample.csv?dl=0  vw_tutorial.pdf
pred.txt               train_medium.csv       vw_vs_xgb.ipynb


In [206]:
!head -n 2 train_medium_vw.vw

1.0 | Unnamed:7881.0 Existing_EMI:0.0 Loan_Amount_Applied:100000.0 Loan_Tenure_Applied:3.0 Monthly_Income:29000.0 Var4:4.0 Var5:11.0 Age:29.0 EMI_Loan_Submitted_Missing:0.0 Interest_Rate_Missing:0.0 Loan_Amount_Submitted_Missing:0.0 Loan_Tenure_Submitted_Missing:0.0 Processing_Fee_Missing:0.0 Device_Type_0:0.0 Device_Type_1:1.0 Filled_Form_0:1.0 Filled_Form_1:0.0 Gender_0:0.0 Gender_1:1.0 Var1_0:0.0 Var1_1:0.0 Var1_2:0.0 Var1_3:0.0 Var1_4:0.0 Var1_5:0.0 Var1_6:0.0 Var1_7:0.0 Var1_8:0.0 Var1_9:0.0 Var1_10:0.0 Var1_11:1.0 Var1_12:0.0 Var1_13:0.0 Var1_14:0.0 Var1_15:0.0 Var1_16:0.0 Var1_17:0.0 Var1_18:0.0 Var2_0:0.0 Var2_1:1.0 Var2_2:0.0 Var2_3:0.0 Var2_4:0.0 Var2_5:0.0 Var2_6:0.0 Mobile_Verified_0:0.0 Mobile_Verified_1:1.0 Source_0:0.0 Source_1:1.0 Source_2:0.0| 
0.0 | Unnamed:189.0 Existing_EMI:21000.0 Loan_Amount_Applied:450000.0 Loan_Tenure_Applied:5.0 Monthly_Income:48000.0 Var4:1.0 Var5:0.0 Age:27.0 EMI_Loan_Submitted_Missing:1.0 Interest_Rate_Missing:1.0 Loan_Amount_Submitted_Missi

In [207]:
#your code here
!vw -d train_medium_vw.vw --passes 10 -c -f model.vw

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train_medium_vw.vw.cache
Reading datafile = train_medium_vw.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       15
0.820184 0.640367            2            2.0   0.0000   0.8002       20
0.570409 0.320634            4            4.0   0.0000   0.0000       18
0.285204 0.000000            8            8.0   0.0000   0.0000       19
0.217465 0.149725           16           16.0   0.0000   0.0413       16
0.108809 0.000152           32           32.0   0.0000   0.0000       15
0.054915 0.001021           64           64.0   0.0000   0.0000       20
0.027685 0.000455          128          128.0   0.0000   0.0000       19
0.013848 0.000012          256          256.0   0

In [209]:
!vw -d test_medium_vw.vw -i model.vw -t -p preds.txt

only testing
predictions = preds.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test_medium_vw.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.001864 0.001864            1            1.0   0.0000   0.0432       18
0.000997 0.000130            2            2.0   0.0000   0.0114       18
0.001139 0.001280            4            4.0   0.0000   0.0506       16
0.002138 0.003138            8            8.0   0.0000   0.0610       20
0.002347 0.002556           16           16.0   0.0000   0.0711       16
0.002225 0.002103           32           32.0   0.0000   0.0444       20
0.016246 0.030267           64           64.0   1.0000   0.0675       16
0.030753 0.045260          128          128.0   0.0000   0.0664       18
0.020058 0.009363          256          256.0   0.0000   0.0001       13
0.019982 0.019907      

In [ ]:
from sklearn import metrics

probs = []
f=open('probs.txt','r')
lines = f.readlines()
for line in lines:
    probs.append(float(line))
    
print('ROC-AUC:', metrics.roc_auc_score(X_test['Disbursed'], probs))

In [211]:
from sklearn import metrics
res = []
f=open('preds.txt','r')
lines = f.readlines()
for line in lines:
    if float(line) < 0:
        res.append(0)
    else:
        res.append(1)
    
print('accuracy:', metrics.accuracy_score(X_test['Disbursed'].values, res))

accuracy: 0.01489312801654792


In [213]:
import xgboost as xgb
train_data = pd.read_csv("train_medium.csv")
test_data = pd.read_csv("test_medium.csv")
y_train = train_data['Disbursed']
y_test = test_data['Disbursed']
X_train = train_data.drop(columns=['Disbursed'])
X_test = test_data.drop(columns=['Disbursed'])
alg = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, max_depth=4)
xgb_m = alg.fit(X_train, y_train)
xgb_pred = xgb_m.predict(X_test)
print('Final prediction:', metrics.accuracy_score(xgb_pred, y_test))

Final prediction: 0.985106871983452


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
